In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib sentencepiece scikit-learn
!pip install tensorboardX
!pip install -q git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/peft.git

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training

import numpy as np
import os
import torch
import torch.nn as nn

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll
CUDA SETUP: CUDA runtime path found: C:\Users\panta\anaconda3\envs\nlp\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll...


c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: C:\Users\panta\anaconda3\envs\nlp did not contain ['cudart64_110.dll', 'cudart64_120.dll'] as expected! Searching further paths...
  warn(msg)
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/usr/bin'), WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/mingw-w64/bin')}
  warn(msg)


In [2]:
BASE_MODEL = "facebook/opt-350m"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = 0

In [ ]:
# BASE_MODEL = "decapoda-research/llama-7b-hf"

# model = LlamaForCausalLM.from_pretrained(
#     BASE_MODEL,
#     load_in_8bit=True,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )

# tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

# tokenizer.pad_token_id = 0

In [3]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 1572864 || all params: 332769280 || trainable%: 0.472659014678278


In [4]:
def tokenize_sample(item, max_seq_length=1024, add_eos_token=True):
        result = tokenizer(
            item,
            truncation=True,
            max_length=max_seq_length,
            padding=True,
        )
        result = {
            "input_ids": result["input_ids"][:-1],
            "attention_mask": result["attention_mask"][:-1],
        }
        if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < max_seq_length
            and add_eos_token
        ):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)
        
        return result

In [5]:
def generate_prompt(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Describe the caption with appropriate tags
### Input:
{data_point['caption_string']}
### Response:
{data_point['tag_string']}"""

In [6]:
from datasets import load_dataset

data = load_dataset("json", data_files=r'dataset/train_data.json')
data = data["train"].train_test_split(test_size=0.05, shuffle=True, seed=42)
data = data.map(lambda x: tokenize_sample(generate_prompt(x)))
data

Found cached dataset json (C:/Users/panta/.cache/huggingface/datasets/json/default-fac367448397b4f6/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at C:\Users\panta\.cache\huggingface\datasets\json\default-fac367448397b4f6\0.0.0\fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e\cache-a753b907c50531b5.arrow and C:\Users\panta\.cache\huggingface\datasets\json\default-fac367448397b4f6\0.0.0\fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e\cache-402f08e87db1eb6b.arrow


Map:   0%|          | 0/18952 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'tag_string', 'input_ids', 'attention_mask'],
        num_rows: 18952
    })
    test: Dataset({
        features: ['caption_string', 'tag_string', 'input_ids', 'attention_mask'],
        num_rows: 998
    })
})

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
tag_list = open(r'dictionaries/tag_dict.txt').read().splitlines()
mlb = MultiLabelBinarizer(classes=tag_list)
mlb.fit([list(tag_list)])

MultiLabelBinarizer(classes=['1girl', 'solo', 'long_hair', 'breasts', 'blush',
                             'looking_at_viewer', 'smile', 'short_hair',
                             'open_mouth', 'bangs', 'blue_eyes',
                             'multiple_girls', 'blonde_hair', 'skirt',
                             'brown_hair', 'large_breasts', 'simple_background',
                             'black_hair', 'eyebrows_visible_through_hair',
                             'thighhighs', 'hair_ornament', 'hat', 'red_eyes',
                             'gloves', 'shirt', 'touhou', '1boy', 'dress',
                             'white_background', 'original', ...])

In [ ]:
from sklearn.metrics import *
from utils import similar_tag


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)[0]
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)[0]

    pred_tags = [x.strip() for x in decoded_preds.split(",")]
    pred_tags_corrected = similar_tag.correct_tags(pred_tags, tag_list)

    tags = [x.strip() for x in decoded_labels.split(",")]

    one_hots_pred = mlb.transform([pred_tags_corrected])
    one_hots_truth = mlb.transform([tags])

    results = {}

    accuracy = accuracy_score(y_true=one_hots_truth, y_pred=one_hots_pred)
    recall = recall_score(
        y_true=one_hots_truth, y_pred=one_hots_pred, average="weighted", zero_division=1
    )
    precision = precision_score(
        y_true=one_hots_truth, y_pred=one_hots_pred, average="weighted", zero_division=1
    )
    f1_micro = f1_score(
        y_true=one_hots_truth, y_pred=one_hots_pred, average="micro", zero_division=1
    )
    f1_macro = f1_score(
        y_true=one_hots_truth, y_pred=one_hots_pred, average="macro", zero_division=1
    )
    f1_weighted = f1_score(
        y_true=one_hots_truth, y_pred=one_hots_pred, average="weighted", zero_division=1
    )

    results["accuracy"] = accuracy
    results["recall"] = recall
    results["precision"] = precision
    results["f1_micro"] = f1_micro
    results["f1_macro"] = f1_macro
    results["f1_weighted"] = f1_weighted

    return {k: round(v, 4) for k, v in results.items()}

In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="outputs/opt-350m",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=32,
    learning_rate=1e-4,
    num_train_epochs=3,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    eval_steps=20,
    logging_steps=5,
    report_to="tensorboard",
    fp16=True,
)

In [10]:
trainer = transformers.Trainer(
    model=model, 
    train_dataset=data['train'],
    eval_dataset=data['test'],
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    args=training_args
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [11]:
trainer.train()

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/444 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\autograd\_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'loss': 3.4971, 'learning_rate': 9.887387387387388e-05, 'epoch': 0.03}
{'loss': 3.2936, 'learning_rate': 9.774774774774775e-05, 'epoch': 0.07}
{'loss': 3.163, 'learning_rate': 9.662162162162163e-05, 'epoch': 0.1}
{'loss': 3.0409, 'learning_rate': 9.54954954954955e-05, 'epoch': 0.14}


  0%|          | 0/125 [00:00<?, ?it/s]

...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.8935216665267944, 'eval_runtime': 142.6651, 'eval_samples_per_second': 6.995, 'eval_steps_per_second': 0.876, 'epoch': 2.7}
{'loss': 1.9605, 'learning_rate': 8.783783783783785e-06, 'epoch': 2.74}
{'loss': 1.9534, 'learning_rate': 7.657657657657658e-06, 'epoch': 2.77}
{'loss': 1.9773, 'learning_rate': 6.531531531531532e-06, 'epoch': 2.8}
{'loss': 1.9688, 'learning_rate': 5.405405405405406e-06, 'epoch': 2.84}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.8915883302688599, 'eval_runtime': 106.0494, 'eval_samples_per_second': 9.411, 'eval_steps_per_second': 1.179, 'epoch': 2.84}


KeyboardInterrupt: 

In [12]:
text = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Describe the caption with appropriate tags
### Input:
Minato Aqua, the virtual youtuber from hololive is wearing a blue maid outfit with maid cap and her pink and blue streaked hair is styled in twintails
### Response:
"""

In [18]:
batch = tokenizer(text, return_tensors='pt').to("cuda")

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=200, no_repeat_ngram_size=3)

print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Describe the caption with appropriate tags
### Input:
Minato Aqua, the virtual youtuber from hololive is wearing a blue maid outfit with maid cap and her pink and blue streaked hair is styled in twintails
### Response:
1girl, blue_hair, blue-eyed_girl, maid_cap, maid-cap, pink_hair_and_blush, pink, blue, pink-eyes, pink_(curly_hair), maid_caps, maid_(curlish), maid-panties, maid(curly), maid(pant)


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("ooferdoodles/text2tags-opt-350m", use_auth_token=True)

In [19]:
model.save_pretrained(r"loras/opt-350m-test")

In [ ]:
%load_ext tensorboard